## Part 1: Preprocessing

In [1]:
!pip install tensorflow
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Input
from scipy.sparse import issparse
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

ImportError: Traceback (most recent call last):
  File "c:\Users\jclar\anaconda3\envs\dev\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [237]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [238]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
print(y_df)

     Attrition              Department
0          Yes                   Sales
1           No  Research & Development
2          Yes  Research & Development
3           No  Research & Development
4           No  Research & Development
...        ...                     ...
1465        No  Research & Development
1466        No  Research & Development
1467        No  Research & Development
1468        No                   Sales
1469        No  Research & Development

[1470 rows x 2 columns]


In [239]:
# Create a list of at least 10 column names to use as X data
column_names = ['Age', 'BusinessTravel', 'PercentSalaryHike', 'DistanceFromHome', 'Education', 'EducationField', 'EnvironmentSatisfaction', 'TotalWorkingYears', 'HourlyRate', 'JobInvolvement']


# Create X_df using your selected columns
X_df = attrition_df[column_names]
print(X_df)

# Show the data types for X_df
dtypes = X_df.dtypes
print(dtypes)



      Age     BusinessTravel  PercentSalaryHike  DistanceFromHome  Education  \
0      41      Travel_Rarely                 11                 1          2   
1      49  Travel_Frequently                 23                 8          1   
2      37      Travel_Rarely                 15                 2          2   
3      33  Travel_Frequently                 11                 3          4   
4      27      Travel_Rarely                 12                 2          1   
...   ...                ...                ...               ...        ...   
1465   36  Travel_Frequently                 17                23          2   
1466   39      Travel_Rarely                 15                 6          1   
1467   27      Travel_Rarely                 20                 4          3   
1468   49  Travel_Frequently                 14                 2          3   
1469   34      Travel_Rarely                 12                 8          3   

     EducationField  EnvironmentSatisfa

In [240]:
from re import X
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)



X_train shape: (1176, 10)
X_test shape: (294, 10)
y_train shape: (1176, 2)
y_test shape: (294, 2)


In [241]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# Ensure both training and testing data have the smae dummy variables
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


In [242]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the testing data using the same scaler
X_test_scaled = scaler.transform(X_test)

# Convert scaled data back to DataFrame (for easier handling)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Display the data
print(X_train_scaled_df.head())
print(X_test_scaled_df.head())


        Age  PercentSalaryHike  DistanceFromHome  Education  \
0 -1.388559          -0.339249          1.440396  -0.863356   
1 -2.040738          -0.066365         -0.522699  -0.863356   
2 -0.845077          -0.339249          1.317703  -0.863356   
3  0.241886           1.570943          0.336155   0.099933   
4 -0.627685          -1.157903          1.317703   0.099933   

   EnvironmentSatisfaction  TotalWorkingYears  HourlyRate  JobInvolvement  \
0                 0.279706          -1.167368   -0.472832       -1.012340   
1                -0.639104          -1.423397    0.309374        0.389912   
2                 1.198515          -0.143254   -1.059487        0.389912   
3                 1.198515          -0.527297   -0.032841        0.389912   
4                -0.639104          -0.143254    1.091580        0.389912   

   BusinessTravel_Non-Travel  BusinessTravel_Travel_Frequently  \
0                  -0.326041                         -0.490414   
1                   3.0670

In [243]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train[['Department']])
y_test_encoded = encoder.transform(y_test[['Department']])


In [244]:
# Create a OneHotEncoder for the Attrition column
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

# Fit the encoder to the training data
encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = encoder.transform(y_train[['Attrition']])
y_test_encoded = encoder.transform(y_test[['Attrition']])



## Create, Compile, and Train the Model

In [245]:
# Import necessary libraries
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Input

# Find the number of columns in the X training data
number_input_features = len(X_train.columns)

# Define the main input layer
main_input = Input(shape=(number_input_features,), name="main_input")

# Create shared layers
shared_layer1 = layers.Dense(units=32, activation='relu')(main_input)
shared_layer2 = layers.Dense(units=16, activation='relu')(shared_layer1)

# Create a branch for Department
department_hidden = layers.Dense(units=64, activation='relu')(shared_layer2)
department_output = layers.Dense(units=2, activation='softmax', name="department_output")(department_hidden)

# Create a branch for Attrition
attrition_hidden = layers.Dense(units=64, activation='relu')(shared_layer2)  # Connect to shared_layer2
attrition_output = layers.Dense(units=2, activation='softmax', name="attrition_output")(attrition_hidden)

# Create the model with one input and two outputs
model = Model(inputs=main_input, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[['accuracy'], ['accuracy']])


# Summarize the model
model.summary()

# Evaluate the model
if issparse(y_test_encoded):
    y_test_encoded_dense = y_test_encoded.toarray()
else:
    y_test_encoded_dense = y_test_encoded  # Already dense, no conversion needed

loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(
    X_test_scaled, [y_test_encoded_dense, y_test_encoded_dense], verbose=0
)


# Print the evaluation results
print("Loss:", loss)
print("Department Loss:", department_loss)
print("Attrition Loss:", attrition_loss)
print("Department Accuracy:", department_accuracy)
print("Attrition Accuracy:", attrition_accuracy)



Model: "functional_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ main_input (InputLayer)   │ (None, 17)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_166 (Dense)         │ (None, 32)             │            576 │ main_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_167 (Dense)         │ (None, 16)             │            528 │ dense_166[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_168 (Dense)         │ (None, 64)             │          1,088 │ dense_167[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_169 (Dense)         │ (None, 64)             │          1,088 │ dense_167[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 2)              │            130 │ dense_168[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │            130 │ dense_169[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,540 (13.83 KB)

 Trainable params: 3,540 (13.83 KB)

 Non-trainable params: 0 (0.00 B)

Loss: 1.4115164279937744
Department Loss: 0.8073211908340454
Attrition Loss: 0.6000092029571533
Department Accuracy: 0.8401360511779785
Attrition Accuracy: 0.24829931557178497


In [248]:
# Train the model
model = model
model.fit(
    X_train_scaled,
    [y_train_encoded.toarray(), y_train_encoded.toarray()],
    epochs=10,
    shuffle=True,
    verbose=2
)





Epoch 1/10
37/37 - 3s - 75ms/step - attrition_output_accuracy: 0.8299 - attrition_output_loss: 0.5318 - department_output_accuracy: 0.7262 - department_output_loss: 0.5820 - loss: 1.1163
Epoch 2/10
37/37 - 0s - 4ms/step - attrition_output_accuracy: 0.8316 - attrition_output_loss: 0.4560 - department_output_accuracy: 0.8316 - department_output_loss: 0.4449 - loss: 0.9000
Epoch 3/10
37/37 - 0s - 4ms/step - attrition_output_accuracy: 0.8316 - attrition_output_loss: 0.4320 - department_output_accuracy: 0.8316 - department_output_loss: 0.4273 - loss: 0.8583
Epoch 4/10
37/37 - 0s - 3ms/step - attrition_output_accuracy: 0.8316 - attrition_output_loss: 0.4194 - department_output_accuracy: 0.8316 - department_output_loss: 0.4160 - loss: 0.8344
Epoch 5/10
37/37 - 0s - 3ms/step - attrition_output_accuracy: 0.8316 - attrition_output_loss: 0.4088 - department_output_accuracy: 0.8325 - department_output_loss: 0.4064 - loss: 0.8169
Epoch 6/10
37/37 - 0s - 2ms/step - attrition_output_accuracy: 0.8316 

In [249]:
# Evaluate the model with the testing data
model_loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(
    X_test_scaled, [y_test_encoded.toarray(), y_test_encoded.toarray()], verbose=2
)

# Print the evaluation results
print("Loss:", model_loss)
print("Department Loss:", department_loss)
print("Attrition Loss:", attrition_loss)
print("Department Accuracy:", department_accuracy)
print("Attrition Accuracy:", attrition_accuracy)





10/10 - 0s - 5ms/step - attrition_output_accuracy: 0.8639 - attrition_output_loss: 0.3304 - department_output_accuracy: 0.8673 - department_output_loss: 0.3269 - loss: 0.6985
Loss: 0.698485255241394
Department Loss: 0.3269296884536743
Attrition Loss: 0.33039337396621704
Department Accuracy: 0.8639456033706665
Attrition Accuracy: 0.8673469424247742


In [250]:
# Print the accuracy for both department and attrition
print("Department Accuracy:", department_accuracy)
print("Attrition Accuracy:", attrition_accuracy)


Department Accuracy: 0.8639456033706665
Attrition Accuracy: 0.8673469424247742


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.